In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
               'price':'{:.2f}','buy_p':'{:.2f}','sell_p':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

cols = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter'.split()
colt = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter number'.split()

name = 'HREIT'

### 1ST Batch

In [2]:
div_year = 2020
div_quarter = 2
ids = '756'

In [3]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,HREIT,12000.0,98617.95,2020-08-14


In [4]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,203,HREIT,2021,1,"30,000",0.1738,"4,692.60",2021-05-25,2021-06-25
1,187,HREIT,2020,4,"30,000",0.1718,"4,638.00",2020-12-04,2020-12-24
2,186,HREIT,2020,3,"30,000",0.1715,"4,630.50",2020-11-24,2020-12-14
3,173,HREIT,2020,2,"12,000",0.1715,"1,852.20",2020-08-18,2020-09-18


In [5]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,HREIT,12000.0,98617.95,0.1738,2085.6,2020-08-14,2021-05-25,2021-06-25,315,2.45,2021,1
1,HREIT,12000.0,98617.95,0.1718,2061.6,2020-08-14,2020-12-04,2020-12-24,132,5.78,2020,4
2,HREIT,12000.0,98617.95,0.1715,2058.0,2020-08-14,2020-11-24,2020-12-14,122,6.24,2020,3
3,HREIT,12000.0,98617.95,0.1715,2058.0,2020-08-14,2020-08-18,2020-09-18,35,21.76,2020,2


In [6]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div1 = df_grp
div1

name
HREIT    8263.2
Name: div_amt, dtype: float64

In [7]:
file_name = 'div-' + name + '.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/div-HREIT.csv',
 '\\Users\\User\\iCloudDrive\\div-HREIT.csv',
 '\\Users\\User\\Dropbox\\div-HREIT.csv')

In [8]:
df_merge[colt].to_csv(output_file)
df_merge[colt].to_csv(data_file)
df_merge[colt].to_csv(box_file)

### Second batch of purchase

In [9]:
div_year = 2020
div_quarter = 3
ids = '764,774,782'

In [10]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,HREIT,18000.0,142815.64,2020-09-01


In [11]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,203,HREIT,2021,1,"30,000",0.1738,"4,692.60",2021-05-25,2021-06-25
1,187,HREIT,2020,4,"30,000",0.1718,"4,638.00",2020-12-04,2020-12-24
2,186,HREIT,2020,3,"30,000",0.1715,"4,630.50",2020-11-24,2020-12-14


In [12]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,HREIT,18000.0,142815.64,0.1738,3128.4,2020-09-01,2021-05-25,2021-06-25,297,2.69,2021,1
1,HREIT,18000.0,142815.64,0.1718,3092.4,2020-09-01,2020-12-04,2020-12-24,114,6.93,2020,4
2,HREIT,18000.0,142815.64,0.1715,3087.0,2020-09-01,2020-11-24,2020-12-14,104,7.59,2020,3


In [13]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div2 = df_grp
div2

name
HREIT    9307.8
Name: div_amt, dtype: float64

In [14]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### End of process

In [15]:
div_year = 2020
div_quarter = 2

In [16]:
sql = """
SELECT id, name, year, quarter, number, ppu, number * ppu AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year, quarter
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,173,HREIT,2020,2,"12,000",0.1715,"2,058.00",2020-08-18,2020-09-18
1,186,HREIT,2020,3,"30,000",0.1715,"5,145.00",2020-11-24,2020-12-14
2,187,HREIT,2020,4,"30,000",0.1718,"5,154.00",2020-12-04,2020-12-24
3,203,HREIT,2021,1,"30,000",0.1738,"5,214.00",2021-05-25,2021-06-25


In [17]:
dividends.div_amt.sum()

17571.0